In [ ]:
import glob
import pandas as pd
import datetime as dt

In [ ]:
t0 = dt.datetime(2010,1,1,1,0) 
t1 = dt.datetime(2019,12,31,23,0) 

In [ ]:
# Reanalysis extracted on Adige (closed to Bronzolo) catchment output data 
path_rea ='/media/windows/data/GFS_models/ECMWF/ERA5Land-reanalysis/derivatives/'
meteo_var_r = ['t2m', 'ptot']

In [ ]:
# Kriging resampling output data 
path_kr = '/media/windows/projects/era5_bias/OLD/comparison/scripts/kriging/AltoAdige/'
meteo_var = ['TMEAN', 'P']

In [ ]:
for var in meteo_var:
    for var_r in meteo_var_r:

        df_k=pd.DataFrame() # dataframe kriging with all data in the directory
        metadata_k=pd.DataFrame() # metadata of the df files

        df=pd.DataFrame() # dataframe reanalysis with all data in the directory
        metadata=pd.DataFrame() # metadata of the df files

        ### assemblo la matrice per fare le medie spaziali e temporali - Reanalysis
        cells_list = []

        if var_r == 't2m':
            f = path_rea  + var_r + '/'
        if var_r == 'ptot':
            f = path_rea  + var_r + '/hourly_ts/'
            
        file_tot = glob.glob( f + '*.csv')
        for file in file_tot:

            data_tot = pd.read_csv(file,index_col=0,parse_dates=True,skiprows=4,names=[os.path.basename(file)[:-4]])
            data_tot_metadata = pd.read_csv(file,index_col=0,parse_dates=True,names=[os.path.basename(file)[:-4]])

            if data_tot_metadata.loc['z'][0] != -999:
                cells_list.append(os.path.basename(file)[:-4])
                #print(file)
                
                data_tot=data_tot[t0:t1]
                data_tot_metadata=data_tot_metadata[0:4]
                data_tot[data_tot == -999] = None
                
                df=pd.concat([df,data_tot],axis=1)
                df.index.name = 'datetime'
                metadata=pd.concat([metadata,data_tot_metadata],axis=1)

        df.index.name = 'datetime'
        
        if var_r == 't2m':
            output = '/media/windows/projects/bias_correction/data/reanalysis_temperature8.csv'
            print("SAVING.." + output)
        if var_r == 'ptot':
            output = '/media/windows/projects/bias_correction/data/reanalysis_precipitation8.csv'
            print("SAVING.." + output)

        df.to_csv(output)

        ### assemblo la matrice per fare le medie spaziali e temporali - kriging
        f = path_kr + var + '/'
        file_tot = glob.glob( f + '*.csv')
        for file in file_tot:

            if os.path.basename(file)[:-4] in cells_list:
                #print(file)
                data_tot = pd.read_csv( file, index_col=0, parse_dates=True, skiprows=4, names=[os.path.basename(file)[:-4]] )
                data_tot_metadata = pd.read_csv( file, index_col=0, parse_dates=True, names=[os.path.basename(file)[:-4]] ) 
                
                data_tot = data_tot[t0:t1]
                data_tot_metadata = data_tot_metadata[0:3]

                df_k=pd.concat([df_k,data_tot],axis=1)
                metadata_k=pd.concat([metadata_k,data_tot_metadata],axis=1)

        df_k.index.name = 'datetime'
        
        if var == 'TMEAN':
            output = '/media/windows/projects/bias_correction/data/kriging_temperature8.csv'
            print("SAVING.." + output)
        if var == 'P':
            output = '/media/windows/projects/bias_correction/data/kriging_precipitation8.csv'
            print("SAVING.." + output)
        
        df_k.to_csv(output)

In [ ]:
data_kr = pd.read_csv( '/media/windows/projects/bias_correction/data/kriging_precipitation8.csv', index_col=0, parse_dates=True )

In [ ]:
df_k.describe()

In [ ]:
data_rea = pd.read_csv( '/media/windows/projects/bias_correction/data/reanalysis_temperature8.csv', index_col=0, parse_dates=True )

In [ ]:
df.describe()